In [1]:
import torch

### A Simple Function

In [2]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [3]:
# Can also create x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
x.grad  # The gradient is None by default

In [5]:
torch.dot(x, x)

tensor(14., grad_fn=<DotBackward0>)

In [4]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [6]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [8]:
x.grad, 4 * x

(tensor([ 0.,  4.,  8., 12.]),
 tensor([ 0.,  4.,  8., 12.], grad_fn=<MulBackward0>))

In [9]:
x.grad == 4 * x

tensor([True, True, True, True])

In [10]:
x.sum()

tensor(6., grad_fn=<SumBackward0>)

In [11]:
x.grad.zero_()  # Reset the gradient
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

### Backward for Non-Scalar Variables

In [18]:
x, x * x

(tensor([0., 1., 2., 3.], requires_grad=True),
 tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>))

In [20]:
x.grad.zero_()
y = x * x
y.backward(gradient=torch.ones(len(y)))  # Faster: y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [22]:
y.sum(), torch.ones(len(y))

(tensor(14., grad_fn=<SumBackward0>), tensor([1., 1., 1., 1.]))

### Detaching Computation

In [64]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [66]:
y, z, u, x.grad

(tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>),
 tensor([ 0.,  1.,  8., 27.], grad_fn=<MulBackward0>),
 tensor([0., 1., 4., 9.]),
 tensor([0., 1., 4., 9.]))

In [67]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

### Gradients and Python Control Flow

In [68]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [79]:
a = torch.randn(size=(), requires_grad=True)
a = torch.tensor(100., requires_grad=True)
d = f(a)
d.backward()
a, d

(tensor(100., requires_grad=True), tensor(1600., grad_fn=<MulBackward0>))

In [80]:
a.grad == d / a, a.grad

(tensor(True), tensor(16.))